# Robot Demo (Moves + Eyes + Camera + TTS)

This is a single notebook demo you can run in front of students.

It tries to show:
- robot movement
- RGB eyes
- camera snapshot
- TTS

If a section fails, it prints the error and carries on.

In [ ]:
import os, time
from pathlib import Path

print('ROS_DOMAIN_ID =', os.environ.get('ROS_DOMAIN_ID'))
print('RMW_IMPLEMENTATION =', os.environ.get('RMW_IMPLEMENTATION'))
print('CYCLONEDDS_URI =', os.environ.get('CYCLONEDDS_URI'))

# If your shell is broken by CYCLONEDDS_URI, you can uncomment this:
# os.environ.pop('CYCLONEDDS_URI', None)

print('Ready')

## Imports

This assumes your repo layout:
- `common/lib/robot.py`
- `common/lib/camera_lib.py`
- `common/lib/tts_lib.py`

And ROS msg types for eyes:
- `ros_robot_controller_msgs/msg/RGBStates`

In [ ]:
import rclpy
from rclpy.node import Node

from ros_robot_controller_msgs.msg import RGBStates, RGBState

from common.lib import robot as robot_lib
from common.lib import camera_lib
from common.lib import tts_lib

print('Imports OK')

## Start ROS node (for RGB publish)

This publishes to `/sonar_controller/set_rgb`.

In [ ]:
if not rclpy.ok():
    rclpy.init()

class DemoNode(Node):
    def __init__(self):
        super().__init__('robot_demo')
        self.rgb_pub = self.create_publisher(RGBStates, '/sonar_controller/set_rgb', 10)

    def set_eye(self, index, r, g, b):
        msg = RGBStates()
        st = RGBState()
        st.index = int(index)
        st.red = int(r)
        st.green = int(g)
        st.blue = int(b)
        msg.states = [st]
        self.rgb_pub.publish(msg)

node = DemoNode()
print('Demo node ready')

## TTS demo

Tries `tts_lib.say()` then `tts_lib.speak()`.

In [ ]:
def speak(text: str):
    if hasattr(tts_lib, 'say'):
        return tts_lib.say(text)
    if hasattr(tts_lib, 'speak'):
        return tts_lib.speak(text)
    raise RuntimeError('tts_lib has no say() or speak()')

try:
    speak('Kia ora team. Watch the robot demo.')
    print('TTS OK')
except Exception as e:
    print('TTS failed:', e)

## Eyes / RGB demo

If this does nothing:
- check sonar_controller is running
- check `smbus2` installed
- check your ROS_DOMAIN_ID in this notebook matches the running stack

In [ ]:
try:
    node.set_eye(0, 255, 0, 0)
    time.sleep(0.5)
    node.set_eye(0, 0, 255, 0)
    time.sleep(0.5)
    node.set_eye(0, 0, 0, 255)
    time.sleep(0.5)
    node.set_eye(0, 0, 0, 0)
    print('RGB publish OK')
except Exception as e:
    print('RGB failed:', e)

## Camera demo

Saves a snapshot to `/tmp/robot_demo/snapshot.jpg` if possible.

In [ ]:
out_dir = Path('/tmp/robot_demo')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'snapshot.jpg'

try:
    if hasattr(camera_lib, 'snapshot'):
        camera_lib.snapshot(str(out_path))
        print('Saved', out_path)
    elif hasattr(camera_lib, 'get_frame'):
        frame = camera_lib.get_frame()
        try:
            import cv2
            cv2.imwrite(str(out_path), frame)
            print('Saved', out_path)
        except Exception as e:
            print('Got frame but could not save (cv2 missing?):', e)
    else:
        print('camera_lib has no snapshot() or get_frame()')
except Exception as e:
    print('Camera failed:', e)

## Robot movement demo

This section is a scaffold because every robot lib differs.

Open `common/lib/robot.py` and adjust the function names below to match your API.

In [ ]:
def try_call(obj, names, *args, **kwargs):
    for n in names:
        if hasattr(obj, n):
            return n, getattr(obj, n)(*args, **kwargs)
    raise AttributeError('None exist: ' + ', '.join(names))

try:
    # Common patterns:
    # - robot_lib.Robot()
    # - robot_lib.robot singleton
    rb = None
    if hasattr(robot_lib, 'Robot'):
        rb = robot_lib.Robot()
    elif hasattr(robot_lib, 'robot'):
        rb = robot_lib.robot
    else:
        rb = robot_lib

    speak('Moving forward')
    try_call(rb, ['forward', 'move_forward', 'drive_forward'], 0.2)
    time.sleep(0.6)

    speak('Turning left')
    try_call(rb, ['turn_left', 'left', 'rotate_left'], 0.3)
    time.sleep(0.6)

    speak('Stopping')
    try_call(rb, ['stop', 'halt', 'brake'])
    print('Move calls OK (if your API matches)')

except Exception as e:
    print('Moves failed:', e)
    print('Open common/lib/robot.py and align names in this cell.')

## Shutdown

In [ ]:
try:
    node.destroy_node()
except Exception:
    pass

try:
    rclpy.shutdown()
except Exception:
    pass

print('Done')